In [1]:
import numpy as np
import math

import gurobipy as gp
from gurobipy import GRB, tuplelist

import pandas as pd

In [2]:
idx_x = np.arange(1,9,1)
set_x = tuplelist([i for i in idx_x])

In [3]:
def variables_ori(m):
    # X
    x = m.addVars(set_x, name='x') 
    x[1].LB, x[1].UB = 100, 10000

    for i in [2,3]:
        x[i].LB, x[i].UB = 1000, 10000

    for i in [4,5,6,7,8]:
        x[i].LB, x[i].UB = 10, 1000

    return x

In [4]:
def objective_ori(m,x):
    sum_x = sum(x[i] for i in [1,2,3])

    m.setObjective(sum_x, GRB.MINIMIZE)

In [5]:
def constraints_ori(m,x):
    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - x[1]*x[6] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(x[2]*x[4] - x[2]*x[7] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(x[3]*x[5] - x[3]*x[8] - 2500*x[5] + 1250000 <= 0)

In [6]:
def constraints_mdt(m,x):
    pairs = tuplelist([(1,6),(2,4),(2,7),(3,5),(3,8)])
    w = m.addVars(pairs, name='w')

    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - w[1,6] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(w[2,4] - w[2,7] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(w[3,5] - w[3,8] - 2500*x[5] + 1250000 <= 0)

    return w,pairs

In [7]:
from collections import Counter
# To find the unique elements from the tuple using the counter

def unique_num(numbers):
    # this will take only unique numbers from the tuple
    return tuple(Counter(numbers).keys())

In [8]:
def relax_mdt(m,x,p=0,P=0):
    w,pairs = constraints_mdt(m,x)
    left_set = unique_num([i[0] for i in pairs])
    right_set = unique_num([i[1] for i in pairs])

    set_l = range(p,P+1)
    set_k = range(10)
    set_kl = tuplelist([(i,j,k,l) for l in set_l for k in set_k for i,j in pairs])
    set_z = tuplelist([(i,k,l) for l in set_l for k in set_k for i in left_set])
    # w,pairs = constraints_mdt(m,x)
    

    # Here, 'left_set' are the variables that are discretized and 'right_set' are the variables that are continuous
    delta_w = m.addVars(pairs, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='delta_w')
    delta_x1 = m.addVars(left_set, lb=0, ub=10**p, name='delta_x1')
    
    # Indexed continuous variables (hat_x_k) and binary variables (z_k)
    hat_x = m.addVars(set_kl, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='hat_x')
    z = m.addVars(set_z, vtype=GRB.BINARY, name='z')

    m.update()
        
    m.addConstrs((w[i,j] == gp.quicksum(gp.quicksum(10**l * k * hat_x[i,j,k,l] for k in set_k) for l in set_l) + delta_w[i,j] for i,j in pairs))

    m.addConstrs((x[i] == gp.quicksum(gp.quicksum(10**l * k * z[i,k,l] for k in set_k) for l in set_l) + delta_x1[i] for i in left_set))

    m.addConstrs((x[j] == gp.quicksum(hat_x[i,j,k,l] for k in set_k) for l in set_l for i,j in pairs))

    m.addConstrs((hat_x[i,j,k,l] >= x[j].LB * z[i,k,l] for i,j,k,l in set_kl))
    m.addConstrs((hat_x[i,j,k,l] <= x[j].UB * z[i,k,l] for i,j,k,l in set_kl))

    m.addConstrs((z.sum(i,'*',l) == 1 for i,k,l in set_z))

    m.addConstrs((delta_w[i,j] >= x[j].LB * delta_x1[i] for i,j in pairs))
    m.addConstrs((delta_w[i,j] <= x[j].UB * delta_x1[i] for i,j in pairs))

    

    return w, delta_w, delta_x1, hat_x, z    # output variables for main program

In [9]:
m = gp.Model('MDT')
x = variables_ori(m)
constraints_mdt(m,x)
objective_ori(m,x)
m.update()
# P = int(math.log10(1000))
w, delta_w, delta_x1, hat_x, z = relax_mdt(m,x,p=2,P=4)

m.update()
m.write('P3.lp')
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 435 rows, 266 columns and 1951 nonzeros
Model fingerprint: 0x2d2b17bd
Variable types: 176 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [3e-03, 9e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+06]
Presolve removed 190 rows and 87 columns
Presolve time: 0.00s
Presolved: 245 rows, 179 columns, 795 nonzeros
Variable types: 116 continuous, 63 integer (63 binary)
Found heuristic solution: objective 9100.0000000

Root relaxation: objective 2.100000e+03, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Boun

In [10]:
math.log10(10000)

4.0

In [11]:
def optimize_problem(p,P):
    m = gp.Model('MDT')
    x = variables_ori(m)
    m.update()
    w = relax_mdt(m,x)
    
    

In [12]:
## Defining the upper bound
m = gp.Model('P3')
x = variables_ori(m)
m.update()
objective_ori(m,x)
constraints_ori(m,x)

m.Params.OutputFlag = 0
m.Params.NonConvex = 2
m.optimize()

x_res = [(i, x[i].X) for i in set_x]
df = pd.DataFrame(x_res, columns=['Index', 'x_upper'])
df = df.set_index('Index')

In [13]:
print(df)

           x_upper
Index             
1       579.306684
2      1359.970668
3      5109.970668
4       182.017700
5       295.601173
6       217.982300
7       286.416526
8       395.601173
